In [101]:
import pandas as pd


In [102]:
results = pd.read_csv('./results.csv', names=['repo', 'acc', '#w', 'samples', 'time', 'model', 'model_address', 'split'])


In [103]:
results['type'] = results['model_address'].apply(lambda s: 'good' if 'good' in s else 'included' if 'included' in s else 'excluded')

In [104]:
results.head(3)

,repo,acc,#w,samples,time,model,model_address,split,type
0,/data/all/data/qooxdoo/qooxdoo,0.55,18,274,24-01-2022_12-25-47,t5-small,t5-small_repo-based-included_21-01-2022_10-30-...,0.5,included
1,/data/all/data/elastic/kibana,0.45,17,113,24-01-2022_12-27-26,t5-small,t5-small_repo-based-included_21-01-2022_10-30-...,0.5,included
2,/data/all/data/emberjs/ember.js,0.58,11,108,24-01-2022_12-28-26,t5-small,t5-small_repo-based-included_21-01-2022_10-30-...,0.5,included


In [105]:
good_5 = results[(results['split'] == 0.5) & (results['type'] == 'good') & (results['repo'] != 'all')]
good_2 = results[(results['split'] == 0.2) & (results['type'] == 'good') & (results['repo'] != 'all')]
inc_2 = results[(results['split'] == 0.2) & (results['type'] == 'included') & (results['repo'] != 'all')]
inc_5 = results[(results['split'] == 0.5) & (results['type'] == 'included') & (results['repo'] != 'all')]
exc_5 = results[(results['split'] == 0.5) & (results['type'] == 'excluded') & (results['repo'] != 'all')]
exc_2 = results[(results['split'] == 0.2) & (results['type'] == 'excluded') & (results['repo'] != 'all')]


In [106]:
print(len(good_5), len(good_2), len(inc_2), len(inc_5), len(exc_2), len(exc_5))

10 10 10 10 10 10


In [107]:
inc_2['acc'].mean(), (inc_2['acc'] * inc_2['samples']).sum() / inc_2['samples'].sum()

(0.509, 0.5161063829787234)

In [108]:
# inc_5['acc'].mean(), (inc_5['acc'] * inc_5['samples']).sum() / inc_5['samples'].sum()

In [109]:
exc_2['acc'].mean(), (exc_2['acc'] * exc_2['samples']).sum() / exc_2['samples'].sum()

(0.493, 0.4946808510638298)

In [110]:
# exc_5['acc'].mean(), (exc_5['acc'] * exc_5['samples']).sum() / exc_5['samples'].sum()

In [111]:
good_2['acc'].mean(), (good_2['acc'] * good_2['samples']).sum() / good_2['samples'].sum()

(0.621, 0.5986808510638297)

In [112]:
# good_5['acc'].mean(), (good_5['acc'] * good_5['samples']).sum() / good_5['samples'].sum()

In [113]:
inc_2

,repo,acc,#w,samples,time,model,model_address,split,type
50,/data/all/data/qooxdoo/qooxdoo,0.53,18,115,25-01-2022_09-42-10,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
51,/data/all/data/elastic/kibana,0.45,17,51,25-01-2022_09-43-03,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
52,/data/all/data/emberjs/ember.js,0.53,11,47,25-01-2022_09-43-44,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
53,/data/all/data/zloirock/core-js,0.24,8,41,25-01-2022_09-44-17,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
54,/data/all/data/Encapsule-Annex/onm,1.00,1,39,25-01-2022_09-44-43,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
55,/data/all/data/sequelize/sequelize,0.35,12,40,25-01-2022_09-45-06,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
56,/data/all/data/dcos/dcos-ui,0.92,9,37,25-01-2022_09-45-43,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
57,/data/all/data/LivelyKernel/LivelyKernel,0.61,12,36,25-01-2022_09-46-15,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
58,/data/all/data/svgdotjs/svg.js,0.13,5,31,25-01-2022_09-46-51,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
59,/data/all/data/foam-framework/foam,0.33,13,33,25-01-2022_09-47-17,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included


In [114]:
exc_2

,repo,acc,#w,samples,time,model,model_address,split,type
20,/data/all/data/qooxdoo/qooxdoo,0.48,18,115,24-01-2022_16-16-11,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
21,/data/all/data/elastic/kibana,0.43,17,51,24-01-2022_16-17-24,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
22,/data/all/data/emberjs/ember.js,0.51,11,47,24-01-2022_16-18-12,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
23,/data/all/data/zloirock/core-js,0.20,8,41,24-01-2022_16-19-13,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
24,/data/all/data/Encapsule-Annex/onm,1.00,1,39,24-01-2022_16-19-55,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
25,/data/all/data/sequelize/sequelize,0.38,12,40,24-01-2022_16-20-31,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
26,/data/all/data/dcos/dcos-ui,0.92,9,37,24-01-2022_16-21-14,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
27,/data/all/data/LivelyKernel/LivelyKernel,0.61,12,36,24-01-2022_16-21-52,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
28,/data/all/data/svgdotjs/svg.js,0.10,5,31,24-01-2022_16-22-35,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded
29,/data/all/data/foam-framework/foam,0.30,13,33,24-01-2022_16-23-19,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,excluded


In [115]:
good_2

,repo,acc,#w,samples,time,model,model_address,split,type
30,/data/all/data/qooxdoo/qooxdoo,0.52,18,115,24-01-2022_17-33-58,t5-small,/scratch/arminz//tmp/finetuned/good_34_qooxdoo...,0.2,good
31,/data/all/data/elastic/kibana,0.49,17,51,24-01-2022_17-36-22,t5-small,/scratch/arminz//tmp/finetuned/good_821_kibana...,0.2,good
32,/data/all/data/emberjs/ember.js,0.55,11,47,24-01-2022_17-38-33,t5-small,/scratch/arminz//tmp/finetuned/good_851_ember....,0.2,good
33,/data/all/data/zloirock/core-js,0.54,8,41,24-01-2022_17-40-51,t5-small,/scratch/arminz//tmp/finetuned/good_815_core-j...,0.2,good
34,/data/all/data/Encapsule-Annex/onm,0.97,1,39,24-01-2022_17-42-46,t5-small,/scratch/arminz//tmp/finetuned/good_80_onm_1.0...,0.2,good
35,/data/all/data/sequelize/sequelize,0.50,12,40,24-01-2022_17-44-59,t5-small,/scratch/arminz//tmp/finetuned/good_14_sequeli...,0.2,good
36,/data/all/data/dcos/dcos-ui,0.86,9,37,24-01-2022_17-46-50,t5-small,/scratch/arminz//tmp/finetuned/good_497_dcos-u...,0.2,good
37,/data/all/data/LivelyKernel/LivelyKernel,0.61,12,36,24-01-2022_17-48-31,t5-small,/scratch/arminz//tmp/finetuned/good_167_Lively...,0.2,good
38,/data/all/data/svgdotjs/svg.js,0.81,5,31,24-01-2022_17-50-33,t5-small,/scratch/arminz//tmp/finetuned/good_221_svg.js...,0.2,good
39,/data/all/data/foam-framework/foam,0.36,13,33,24-01-2022_17-52-38,t5-small,/scratch/arminz//tmp/finetuned/good_282_foam_1...,0.2,good


In [116]:
results[(results['repo'] == 'all') & (results['split'] == 0.2)]

,repo,acc,#w,samples,time,model,model_address,split,type
60,all,0.40,48,3374,25-01-2022_10-31-49,t5-small,/scratch/arminz/training/t5-small_repo-based-i...,0.2,included
61,all,0.38,48,3374,25-01-2022_10-35-54,t5-small,/scratch/arminz/training//t5-small_repo-based_...,0.2,excluded


In [121]:
((0.02 * 3374) - (470 * 0.022)) / (3374 - 470)

0.019676308539944905